# Semantic Layer on CSV

In this notebook, we will show how to create a semantic layer on a CSV file.
The semantic layer works as a bridge between the raw data and the natural language layer.

### Why use a Semantic Layer?
- Adds context and meaning to data columns
- Makes it easier for the large language model to understand context
- Set once, use across multiple sessions

## Import PandasAI

In [3]:
import pandasai as pai


## Read raw data

For this example, we will use a small dataset of heart disease patients from [Kaggle](https://www.kaggle.com/datasets/arezaei81/heartcsv).

In [5]:

# Load the heart disease dataset
# Using relative path: go up two levels (../..) to examples, then into data folder
file_df = pai.read_csv("../../data/heart.csv")
# Display the first few rows
file_df.head()


,Age,Sex,ChestPainType,RestingBP,Cholesterol,FastingBS,RestingECG,MaxHR,ExerciseAngina,Oldpeak,ST_Slope,HeartDisease
0,40,M,ATA,140,289,0,Normal,172,N,0.0,Up,0
1,49,F,NAP,160,180,0,Normal,156,N,1.0,Flat,1
2,37,M,ATA,130,283,0,ST,98,N,0.0,Up,0
3,48,F,ASY,138,214,0,Normal,108,Y,1.5,Flat,1
4,54,M,NAP,150,195,0,Normal,122,N,0.0,Up,0


## Create the Semantic Layer

Requirements for the semantic layer:
- `path`: Must be in format 'organization/dataset'
-  `df`: A dataframe
- `description`: Brief overview of the dataset
- `columns`: List of dictionaries with format:
  ```python
  {
      "name": "column_name",
      "type": "column_type",  # string, number, date, datetime
      "description": "column_description"
  }
  ```

In [6]:
try:
    dataset = pai.create(path="organization/heart",
                        description="Heart Disease Dataset",
                        df=file_df,
                        columns=[
                            {
                                "name": "Age",
                                "type": "integer",
                                "description": "Age of the patient in years"
                            },
                            {
                                "name": "Sex",
                                "type": "string",
                                "description": "Gender of the patient (M: Male, F: Female)"
                            },
                            {
                                "name": "ChestPainType",
                                "type": "string",
                                "description": "Type of chest pain (ATA: Atypical Angina, NAP: Non-Anginal Pain, ASY: Asymptomatic, TA: Typical Angina)"
                            },
                            {
                                "name": "RestingBP",
                                "type": "integer",
                                "description": "Resting blood pressure in mm Hg"
                            },
                            {
                                "name": "Cholesterol",
                                "type": "integer",
                                "description": "Serum cholesterol in mg/dl"
                            },
                            {
                                "name": "FastingBS",
                                "type": "integer",
                                "description": "Fasting blood sugar (1: if FastingBS > 120 mg/dl, 0: otherwise)"
                            },
                            {
                                "name": "RestingECG",
                                "type": "string",
                                "description": "Resting electrocardiogram results (Normal, ST: having ST-T wave abnormality, LVH: showing probable or definite left ventricular hypertrophy)"
                            },
                            {
                                "name": "MaxHR",
                                "type": "integer",
                                "description": "Maximum heart rate achieved"
                            },
                            {
                                "name": "ExerciseAngina",
                                "type": "string",
                                "description": "Exercise-induced angina (Y: Yes, N: No)"
                            },
                            {
                                "name": "Oldpeak",
                                "type": "float",
                                "description": "ST depression induced by exercise relative to rest"
                            },
                            {
                                "name": "ST_Slope",
                                "type": "string",
                                "description": "Slope of the peak exercise ST segment (Up, Flat, Down)"
                            },
                            {
                                "name": "HeartDisease",
                                "type": "integer",
                                "description": "Target variable - Heart disease presence (1: heart disease, 0: normal)"
                            }
                        ])
except ValueError:
    print('Dataset already exists at path: organization/heart')

Dataset already exists at path: organization/heart


## Load Semantic Dataframe

Once you have saved the dataframe with its semantic layer, you can load it in any session using the `load()` method. This allows you to:
- Maintain data context across sessions
- Ask questions about your data in natural language
- Generate more accurate analysis and visualizations

In [4]:

# Load the semantically enhanced dataset
df = pai.load("organization/heart")


Dataset loaded successfully.


## Chat with your dataframe

You can now ask questions about your data in natural language to your dataframe using the `chat()` method. PandasAI can be used with several LLMs. For the purpose of this example, we are using LiteLLM.

In [5]:
import os
from pandasai_litellm.litellm import LiteLLM
# Initialize LiteLLM with your OpenAI model
api_key = os.getenv("OPENAI_API_KEY", "your-api-key")
llm = LiteLLM(model="gpt-5-mini", api_key=api_key)
pai.config.set({
    "llm": llm,
    "save_logs": True,
    "max_retries": 3
})

# Configure PandasAI to use this LLM
response = pai.chat("What is the correlation between age and cholesterol?",df)
print(response)
response = df.chat("What is the correlation between age and cholesterol?")
print(response)


-0.0953
-0.0953
